### Import Libraries

In [128]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import csv # library to handle csv files

# import geocoder as geocoder
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. London neighbbourhood informatio - Scraping data from Wikipedia

In [129]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_areas_of_London')
raw = dfs[1]
raw.head()

df1 = raw.drop(df.index[0])
df1.columns = ['Location','London_borough','Post town','Postcode district','Dial Code','OS grid ref']
df1.head()

,Location,London_borough,Post town,Postcode district,Dial Code,OS grid ref
1,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
2,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
3,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
4,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
5,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [209]:
# Drop the unnecessary information and clean data
df = df1.drop(['Dial Code','OS grid ref'], axis = 1)
df = df.reindex()
df.head()

,Location,London_borough,Post town,Postcode district
1,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2
2,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
3,Addington,Croydon[8],CROYDON,CR0
4,Addiscombe,Croydon[8],CROYDON,CR0
5,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [210]:
df.shape

(533, 4)

#### From foursquare API get logitude and altitude of London Neighbourhood

### 2. Must-visit places information - from one (or multiple) review article https://foursquare.com/gmissawa/list/places-to-visit-in-london

In [186]:
#Get the data with beautifulsoup
source = requests.get('https://foursquare.com/gmissawa/list/places-to-visit-in-london').text
soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [187]:
# Scraping the venue information
match = soup.find('div', class_='venueInfo')
#print(match.prettify())

In [197]:
# Scraping trial, to find all the information in the first entry
# venue_name = match.find('p', class_ = 'venueName')
# rating = match.find('div', class_ = 'rating')
# category = match.find('span', class_ = "unlinkedCategory")

# print (venue_name.text)
# print(rating.text)
# print(category.text)

In [211]:
# Save as csv
csv_file = open('London_venue.csv','w')
csv_writer = csv.writer(csv_file, lineterminator='\n')
csv_writer.writerow(['Venue Name', 'Category', 'Rating', 'Venue ID'])

# Loop through all the venues and save the inforamtion into a list
for match in soup.find_all('div', class_='venueInfo'):
    
    venue_name = match.find('p', class_ = 'venueName').text
    category = match.find('span', class_ = "unlinkedCategory").text
#     print (venue_name)
#     print(category)
    
    try:
        rating = match.find('div', class_ = 'rating').text
#         print(rating)
        venue_src = match.find('a')['href']
        venue_id = venue_src.split('/')[3]
#         print(venue_id)
    except Exception as e:
        rating = 'NaN'
#         print('NaN')
        veneue_id = 'NaN'
    # Save as csv
    csv_writer.writerow([venue_name, category, rating, venue_id])

csv_file.close()


In [212]:
# Read csv file as pandas dataframe
df_ln = pd.read_csv("London_venue.csv", encoding = "ISO-8859-1")
df_ln.head()

,Venue Name,Category,Rating,Venue ID
0,1. Big Ben (Elizabeth Tower),Monument / Landmark,9.3,4ac518cef964a520f6a520e3
1,2. The London Eye,Scenic Lookout,9.2,4ac518cef964a52021a620e3
2,3. Trafalgar Square,Plaza,9.2,4ac518cef964a520f9a520e3
3,4. Tower of London,Castle,9.2,4ac518cef964a520f7a520e3
4,5. Westminster Abbey,Church,9.2,4ac518cdf964a520eea520e3


In [213]:
df_ln.shape

(136, 4)

#### From Foursquare API, get the logitude and altitude according to VeneueID
